# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tralee,52.2704,-9.7026,16.14,82,40,2.57,IE,1697735577
1,1,vila franca do campo,37.7167,-25.4333,20.23,99,16,7.64,PT,1697735577
2,2,spittal an der drau,46.8000,13.5000,12.94,100,100,1.09,AT,1697735578
3,3,hasaki,35.7333,140.8333,17.38,69,62,6.35,JP,1697735578
4,4,invercargill,-46.4000,168.3500,10.57,96,96,3.58,NZ,1697735578


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)
# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()

# Display sample data
city_ideal


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,rawson,-43.3002,-65.1023,26.16,17,0,3.60,AR,1697735357
75,75,fort bragg,35.1390,-79.0060,22.68,45,0,1.54,US,1697735590
196,196,newman,37.3138,-121.0208,24.57,59,0,0.45,US,1697735625
226,226,berryville,36.3648,-93.5680,21.23,56,0,4.11,US,1697735633
234,234,al bawiti,28.3492,28.8659,24.73,28,0,3.54,EG,1697735634
286,286,at taj,24.2049,23.2857,24.56,19,0,2.96,LY,1697735641
296,296,el granada,37.5027,-122.4694,26.66,63,0,1.54,US,1697735643
324,324,masjed soleyman,31.9364,49.3039,25.75,41,0,1.03,IR,1697735647
346,346,crane,31.3974,-102.3501,22.95,26,0,0.43,US,1697735655
402,402,kuruman,-27.4524,23.4325,26.11,16,0,3.03,ZA,1697735674


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
12,rawson,AR,-43.3002,-65.1023,17,
75,fort bragg,US,35.1390,-79.0060,45,
196,newman,US,37.3138,-121.0208,59,
226,berryville,US,36.3648,-93.5680,56,
234,al bawiti,EG,28.3492,28.8659,28,
286,at taj,LY,24.2049,23.2857,19,
296,el granada,US,37.5027,-122.4694,63,
324,masjed soleyman,IR,31.9364,49.3039,41,
346,crane,US,31.3974,-102.3501,26,
402,kuruman,ZA,-27.4524,23.4325,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rawson - nearest hotel: Hotel Deportivo
fort bragg - nearest hotel: Airborne Inn Lodging
newman - nearest hotel: No hotel found
berryville - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
at taj - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
masjed soleyman - nearest hotel: هتل جهانگردی
crane - nearest hotel: No hotel found
kuruman - nearest hotel: Palermo Inn
darya khan - nearest hotel: No hotel found
cambria - nearest hotel: Olallieberry Inn
kayenta - nearest hotel: Wetherill Inn
holualoa - nearest hotel: Kona Hotel
kot ghulam muhammad - nearest hotel: No hotel found
butte - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,rawson,AR,-43.3002,-65.1023,17,Hotel Deportivo
75,fort bragg,US,35.1390,-79.0060,45,Airborne Inn Lodging
196,newman,US,37.3138,-121.0208,59,No hotel found
226,berryville,US,36.3648,-93.5680,56,No hotel found
234,al bawiti,EG,28.3492,28.8659,28,Old Oasis Hotel
286,at taj,LY,24.2049,23.2857,19,No hotel found
296,el granada,US,37.5027,-122.4694,63,Beach House
324,masjed soleyman,IR,31.9364,49.3039,41,هتل جهانگردی
346,crane,US,31.3974,-102.3501,26,No hotel found
402,kuruman,ZA,-27.4524,23.4325,16,Palermo Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)